In [22]:
import pandas as pd 

catalog_path = '/Users/tkiker/Documents/GitHub/khcloudnet/data/declassiii-catalog.csv'

df = pd.read_csv(catalog_path, encoding="utf-8")
df = df[df["Camera Resolution"] == "2 to 4 feet"]

len_high_res = len(df.index)
len_high_res

568805

## NOTE TO SELF: NEED TO INCORPORATE IGNORING OF STRIPS > 10x10 degrees into the filtering!

In [23]:
columns_to_check = ["Center Latitude dec", "Center Longitude dec", "NW Corner Lat dec", "SE Corner Long dec"]

# Apply the filter to remove rows with zero values in any of the specified columns
df = df[(df[columns_to_check] != 0).all(axis=1)]

len_filtered = len(df.index)
len_filtered

535834

In [24]:
# As this is an initial study, reduce dependence on human 

def filter_no_snow_dates(row):
    lat = row["Center Latitude dec"]
    date = pd.to_datetime(row["Acquisition Date"])
    month = date.month

    # Define no-snow period by latitude band
    if lat > 66:  # Arctic
        return month == 7  # July only
    elif lat > 45:  # Mid-latitudes
        return month in [6, 7, 8]  # June to August
    else:  # Lower latitudes
        return month in [5, 6, 7, 8, 9]  # May to September

df = df[df.apply(filter_no_snow_dates, axis=1)]

len_no_snow = len(df.index)
len_no_snow

237642

In [25]:
# Find duplicate Entity IDs
duplicate_ids = df[df.duplicated(subset='Entity ID', keep=False)]['Entity ID']

# Print a few duplicate IDs (e.g., first 10)
print(duplicate_ids.head(10).to_list())

df = df.drop_duplicates(subset='Entity ID', keep='first').reset_index(drop=True)

len_no_duplicates = len(df.index)
len_no_duplicates

['D3C1201-200086A021', 'D3C1201-200086A022', 'D3C1201-200086A023', 'D3C1201-200086A024', 'D3C1201-200086A025', 'D3C1201-200086A026', 'D3C1201-200086A027', 'D3C1201-200086A028', 'D3C1201-200086A029', 'D3C1201-200086A030']


236294

In [26]:
df.to_csv('/Users/tkiker/Documents/GitHub/khcloudnet/data/filtered_key_df.csv', index=False)

In [32]:
# interesting statistic for paper:

import pandas as pd

df = pd.read_csv('/Users/tkiker/Documents/GitHub/khcloudnet/data/filtered_key_df.csv')  

# Extract Prefix and Suffix
df['Prefix'] = df['Entity ID'].str[:-4]   
df['Suffix'] = df['Entity ID'].str[-4]

df['Conjugate Suffix'] = df['Suffix'].map({'A': 'F', 'F': 'A'})
df['Conjugate ID'] = df['Prefix'] + df['Conjugate Suffix'] + df['Entity ID'].str[-3:]

# Self-merge to find conjugate pairs
merged_df = df.merge(df[['Entity ID']], left_on='Conjugate ID', right_on='Entity ID', how='left', indicator=True)

# Rows where `_merge` is 'both' have conjugates; 'left_only' means no conjugate was found
has_conjugate = merged_df['_merge'] == 'both'

# Count results
count_A_with_conjugate = ((merged_df['Suffix'] == 'A') & has_conjugate).sum()
count_A_without_conjugate = ((merged_df['Suffix'] == 'A') & ~has_conjugate).sum()

count_F_with_conjugate = ((merged_df['Suffix'] == 'F') & has_conjugate).sum()
count_F_without_conjugate = ((merged_df['Suffix'] == 'F') & ~has_conjugate).sum()

# Print the results
print(f"Count of rows with 'A' suffix that have a conjugate 'F': {count_A_with_conjugate}")
print(f"Count of rows with 'A' suffix that do NOT have a conjugate 'F': {count_A_without_conjugate}")
print(f"Count of rows with 'F' suffix that have a conjugate 'A': {count_F_with_conjugate}")
print(f"Count of rows with 'F' suffix that do NOT have a conjugate 'A': {count_F_without_conjugate}")

Count of rows with 'A' suffix that have a conjugate 'F': 100803
Count of rows with 'A' suffix that do NOT have a conjugate 'F': 9658
Count of rows with 'F' suffix that have a conjugate 'A': 100803
Count of rows with 'F' suffix that do NOT have a conjugate 'A': 25030
